## PTC and MAD-X

We will compare PTC adn MAD-X.



In [7]:
# as usual we will use MAD-X via python
from cpymad.madx import Madx

# standard packages and modules
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# standard configurations
get_ipython().magic('matplotlib inline')
%config InlineBackend.figure_format = 'retina' # retina display

# This functions will be used for the printing/plotting (nothing fundamental)
def showTunes():
    print(f'Q1 = {madx.table.summ.q1[0]:1.6}')
    print(f'Q2 = {madx.table.summ.q2[0]:1.6}')
    
def showChroma():
    print(f'DQ1 = {madx.table.summ.dq1[0]:1.6}')
    print(f'DQ2 = {madx.table.summ.dq2[0]:1.6}')

def plotLatticeSeries(ax, series, height=1., v_offset=0., color='r',alpha=0.5,lw=1):
    aux=series
    ax.add_patch(
    patches.Rectangle(
        (aux.s-aux.l, v_offset-height/2.),   # (x,y)
        aux.l,          # width
        height,          # height
        color=color, alpha=alpha,lw=lw
    )
    )
    return;

In [8]:
def BeamParameters(pc_GeV, en_x_m=5e-6, en_y_m=5e-6, deltap_p=1e-3, verbose=True):
    Brho_Tm=3.3356*pc_GeV # Tm, beam rigidity (approx)
    E_0_GeV=0.9382720813 # GeV, rest mass energy
    E_tot_GeV=np.sqrt(pc_GeV**2+E_0_GeV**2)
    E_kin_GeV=E_tot_GeV-E_0_GeV
    gamma_r=E_tot_GeV/E_0_GeV
    beta_r=pc_GeV/E_tot_GeV
    eg_x_m=en_x_m/gamma_r/beta_r
    eg_y_m=en_y_m/gamma_r/beta_r


    if verbose:
        print(f'''Particle type: proton
        Beam momentum= {pc_GeV:2.3f} GeV/c
        normalized x-emittance= {en_x_m*1e6:2.3f} mm mrad
        normalized y-emittance= {en_y_m*1e6:2.3f} mm mrad
        deltap/p= {deltap_p} 
        -> Beam total energy= {E_tot_GeV:2.3f} GeV
        -> Beam kinetic energy= {E_kin_GeV:2.3f} GeV
        -> Beam rigidity= {Brho_Tm:2.3f} Tm
        -> relativistic beta= {beta_r:2.5f}
        -> relativistic gamma= {gamma_r:2.3f}
        -> geometrical x-emittance= {eg_x_m*1e6:2.3f} mm mrad
        -> geometrical y-emittance= {eg_y_m*1e6:2.3f} mm mrad
        ''')
    return {'pc_GeV': pc_GeV,'Brho_Tm': Brho_Tm,'E_0_GeV': E_0_GeV, 'E_tot_GeV':E_tot_GeV,
            'E_kin_GeV': E_kin_GeV, 'gamma_r': gamma_r, 'beta_r':beta_r, 'en_x_m':en_x_m, 'en_y_m':en_y_m,
            'eg_x_m':eg_x_m, 'eg_y_m':eg_y_m, 'deltap_p':deltap_p}

print('====== Injection Energy ======')
beamFB=BeamParameters(1.9, en_x_m=5e-6, en_y_m=5e-6, deltap_p=2e-3)

print('======== Top Energy ========')
beamFT=BeamParameters(19, en_x_m=5e-6, en_y_m=5e-6, deltap_p=2e-4)


====== Injection Energy ======
Particle type: proton
        Beam momentum= 1.900 GeV/c
        normalized x-emittance= 5.000 mm mrad
        normalized y-emittance= 5.000 mm mrad
        deltap/p= 0.002 
        -> Beam total energy= 2.119 GeV
        -> Beam kinetic energy= 1.181 GeV
        -> Beam rigidity= 6.338 Tm
        -> relativistic beta= 0.89663
        -> relativistic gamma= 2.258
        -> geometrical x-emittance= 2.469 mm mrad
        -> geometrical y-emittance= 2.469 mm mrad
        
======== Top Energy ========
Particle type: proton
        Beam momentum= 19.000 GeV/c
        normalized x-emittance= 5.000 mm mrad
        normalized y-emittance= 5.000 mm mrad
        deltap/p= 0.0002 
        -> Beam total energy= 19.023 GeV
        -> Beam kinetic energy= 18.085 GeV
        -> Beam rigidity= 63.376 Tm
        -> relativistic beta= 0.99878
        -> relativistic gamma= 20.275
        -> geometrical x-emittance= 0.247 mm mrad
        -> geometrical y-emittance= 0.247 m

In [9]:
circum_m=500. # m, machine circumference
ncell=25.
l_quad_m=.5 # m
l_dip_m=3.5 # m
lcell_m=circum_m/ncell; # m

f_m=lcell_m/(2*np.sqrt(2)) # This will give pi/2 phase advance in thin lens approximation (no dipoles)

nQuadrupoles=2*ncell
nDipoles=4*ncell # four dipoles per cell
angleOfDipole_rad=2*np.pi/nDipoles 
fieldOfDipole_T= angleOfDipole_rad*beamFT['Brho_Tm']/l_dip_m
gradientOfQuadrupole_T_m= 1/f_m*beamFT['Brho_Tm']/l_quad_m
r_quadrupole_m= 0.065;
v_gap_dipole_m= 0.065;
h_gap_dipole_m= 0.09;

print(f'''Machine circumference: {circum_m} m
Number of FODO cells= {ncell}
Quadrupole length= {l_quad_m} m
Dipole length= {l_dip_m} m
Number of quadrupoles= {nQuadrupoles}
Number of dipoles= {nDipoles}
Quadrupole focal length= {f_m:2.3f} m
Radius of the quadrupole's aperture =  {r_quadrupole_m*1000} mm
Horizontal half-gap of the dipoles=  {h_gap_dipole_m*1000} mm
Vertical half-gap of the dipoles=  {v_gap_dipole_m*1000} mm

-> Length of FODO= {lcell_m} m
-> Angle of dipole = {angleOfDipole_rad*1000: 2.3f} mrad
-> Field of dipole = {fieldOfDipole_T: 2.3f} T
-> K1 of quadrupole = {1/f_m/l_quad_m: 2.3f} 1/m^2
-> Gradient of quadrupole = {gradientOfQuadrupole_T_m: 2.3f} T/m
-> Field on the tip of the quadrupole = {gradientOfQuadrupole_T_m*r_quadrupole_m: 2.3f} T
''')

Machine circumference: 500.0 m
Number of FODO cells= 25.0
Quadrupole length= 0.5 m
Dipole length= 3.5 m
Number of quadrupoles= 50.0
Number of dipoles= 100.0
Quadrupole focal length= 7.071 m
Radius of the quadrupole's aperture =  65.0 mm
Horizontal half-gap of the dipoles=  90.0 mm
Vertical half-gap of the dipoles=  65.0 mm

-> Length of FODO= 20.0 m
-> Angle of dipole =  62.832 mrad
-> Field of dipole =  1.138 T
-> K1 of quadrupole =  0.283 1/m^2
-> Gradient of quadrupole =  17.926 T/m
-> Field on the tip of the quadrupole =  1.165 T



In [48]:
madx = Madx(stdout=True)
madx.input(f'''
circum={circum_m};
ncell ={ncell}; !Number of cells 
lcell = {lcell_m};
lq = {l_quad_m}; !Length of a quadrupole
ldip = {l_dip_m};
!element definitions;
!define bending magnet as multipole 
!we have 4 bending magnets per cell
!mb:multipole,knl={{2.0*pi/(4*ncell)}};

mb: sbend, l=ldip, angle=2.0*pi/(4*ncell), apertype=ellipse, aperture= {{{h_gap_dipole_m}, {v_gap_dipole_m}}};

f={f_m};

!define quadrupoles as multipoles 
qf: multipole,knl:={{0,1/f+qtrim_f}}; 
qd: multipole,knl:={{0,-1/f+qtrim_d}};
qf: quadrupole, l=lq, K1:=1/f/lq  + qtrim_f/lq, apertype=ellipse, aperture= {{{r_quadrupole_m}, {r_quadrupole_m}}}; 
qd: quadrupole, l=lq, K1:=-1/f/lq + qtrim_d/lq, apertype=ellipse, aperture= {{{r_quadrupole_m}, {r_quadrupole_m}}};

!define the sextupoles as multipole
lsex = 0.00001; ! dummy length, only used in the sequence;

!ATTENTION: must use knl:= and NOT knl= to match later! 
msf: multipole, knl:={{0,0,ksf}};
msd: multipole, knl:={{0,0,ksd}};

!sequence declaration;
!I switch off the warning to limit the output
!use this option with moderation (I switch it back in after the sequence)
option, warn=false;
cas19: sequence, refer=centre, l=circum;
   start_machine: marker, at = 0;
   n = 1;
   while (n < ncell+1) {{
    qf: qf,   at=(n-1)*lcell+ lq/2.0;
    msf: msf, at=(n-1)*lcell + lsex/2.0+lq/1.0;
    mb: mb,   at=(n-1)*lcell + 0.15*lcell;
    mb: mb,   at=(n-1)*lcell + 0.35*lcell;
    qd: qd,   at=(n-1)*lcell + 0.50*lcell+ lq/2.0;
    msd: msd, at=(n-1)*lcell + 0.50*lcell + lsex/2.0+lq/1.0;
    mb: mb,   at=(n-1)*lcell + 0.65*lcell;
    mb: mb,   at=(n-1)*lcell + 0.85*lcell;
    n = n + 1;
}}
end_machine: marker at=circum;
endsequence;
option, warn=true;

!define the beam and its properties
beam, particle = proton, sequence=cas19, energy = {beamFB['E_tot_GeV']}, exn={beamFB['en_x_m']}, eyn={beamFB['en_y_m']},sige={beamFB['en_y_m']};

use, sequence=cas19;
select, flag=twiss, column=apertype, aper_1, aper_2;

ksf=0;
ksd=0;
twiss;
''')
myTwiss=madx.table.twiss.dframe()
showTunes()

Q1 = 6.26177
Q2 = 6.11714


In [49]:
madx.input(f'''
ptc_create_universe;
ptc_create_layout,model=2,method=6,nst=10,exact;
ptc_start, x= 3e-3, px=0, y= 3e-3, py=0;
ptc_start, x= 6e-3, px=0, y= 6e-3, py=0;
ptc_start, x= 9e-3, px=0, y= 9e-3, py=0;
ptc_track,icase=4, closed_orbit,dump,turns=1000;! ffile=1, norm_no=1;
ptc_track_end; 
ptc_end;
''')

True

In [43]:
list(madx.table)

['summ',
 'twiss',
 'errors_dipole',
 'errors_field',
 'errors_total',
 'tracksumm',
 'track.obs0001.p0001',
 'track.obs0001.p0002',
 'track.obs0001.p0003']

In [47]:
madx.table['track.obs0001.p0003'].dframe()

,number,turn,x,px,y,py,t,pt,s,e
#e,3.0,0.0,0.009000,0.000000,0.009000,0.000000,0.0,0.0,0.0,2.119046
#e,3.0,1.0,-0.021496,-0.001753,0.009577,-0.001189,0.0,0.0,0.0,2.119046
#e,3.0,2.0,-0.005849,0.000257,0.005193,-0.001762,0.0,0.0,0.0,2.119046
#e,3.0,3.0,0.022353,0.001714,-0.001880,-0.001424,0.0,0.0,0.0,2.119046
#e,3.0,4.0,0.002509,-0.000513,-0.007980,-0.000347,0.0,0.0,0.0,2.119046
#e,3.0,5.0,-0.022732,-0.001639,-0.009948,0.000909,0.0,0.0,0.0,2.119046
#e,3.0,6.0,0.000823,0.000754,-0.006762,0.001694,0.0,0.0,0.0,2.119046
#e,3.0,7.0,0.022603,0.001527,-0.000076,0.001603,0.0,0.0,0.0,2.119046
#e,3.0,8.0,-0.004200,-0.000982,0.006650,0.000680,0.0,0.0,0.0,2.119046
#e,3.0,9.0,-0.021990,-0.001382,0.009933,-0.000593,0.0,0.0,0.0,2.119046
